# Animation

PyPSA supports animating network plots over snapshots using
[`n.plot.animate()`][pypsa.plot.PlotAccessor.animate], which returns a
[`matplotlib.animation.FuncAnimation`](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html)
object.

Time-varying parameters (e.g. bus sizes, line flows, line widths) are passed as
`pd.DataFrame` objects with snapshots as the index. Static parameters are passed
as usual (scalars, strings, or `pd.Series`).

Let's start by loading an optimizing the `ac_dc_meshed` example network:

In [ ]:
import pypsa

n = pypsa.examples.ac_dc_meshed()
n.optimize(log_to_console=False, include_objective_constant=False)

From the solved network, we can extract the bus sizes and line loadings over time as `pd.DataFrame` objects:


In [ ]:
bus_size = (
    n.statistics.energy_balance(groupby=["bus", "carrier"], groupby_time=False)
    .droplevel("component")
    .drop("DC", level="carrier")
    .drop("AC", level="carrier")
).T

link_loading = n.links_t.p0.abs().div(n.links.p_nom_opt)

line_loading = n.lines_t.p0.abs().div(n.lines.s_nom_opt)


If we pick any snapshot `i`, we can plot the network at that time using the time-varying parameters:

In [ ]:
i = 2

n.plot(
    bus_size=bus_size.iloc[i] / 2000,
    bus_split_circle=True,
    link_flow=n.links_t.p0.iloc[i] / 75,
    line_flow=n.lines_t.p0.iloc[i] / 75,
    link_color=link_loading.iloc[i],
    line_color=line_loading.iloc[i],
    link_width=n.links_t.p0.abs().iloc[i] / 100,
    line_width=n.lines_t.p0.abs().iloc[i] / 100,
    margin=0.25,
    figsize=(10, 8),
);

For the animation, we can pass the time-varying parameters directly to
[`n.plot.animate()`][pypsa.plot.PlotAccessor.animate]. This function takes the same
parameters as [`n.plot()`][pypsa.plot.PlotAccessor.plot], but with the ability to
pass `pd.DataFrame` objects for time-varying parameters, and additional parameters
for controlling the animation output, such as `path`, `writer`, `dpi`, and `fps`:

In [ ]:
n.plot.animate(
    bus_size=bus_size / 1000,
    bus_split_circle=True,
    link_flow=n.links_t.p0.iloc[i] / 75,
    line_flow=n.lines_t.p0.iloc[i] / 75,
    link_color=link_loading,
    line_color=line_loading,
    link_width=n.links_t.p0.abs() / 100,
    line_width=n.lines_t.p0.abs() / 100,
    margin=0.25,
    figsize=(10, 8),
    path="animation.mp4",
    writer="ffmpeg",
    dpi=300,
    fps=5,
);

The resulting animation is saved as `animation.mp4` in the current working directory.